<a href="https://colab.research.google.com/github/Rushikesh419/RFM-analysis/blob/main/RFM_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**RFM Analysis**

RFM analysis is a marketing technique used to quantitatively rank and group customers based on the recency, frequency and monetary total of their recent transactions to identify the best customers and perform targeted marketing campaigns.

RFM is a method used for analyzing customer value. It is commonly used in database marketing and direct marketing and has received particular attention in retail and professional services industries

**RFM **
* Recency (R): How recently customers have made their purchases.
* Frequency (F): How often customers have made their purchases.
* Monetary (M): How much money customers have paid for their purchases.




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import os
import warnings

data=pd.read_csv('/content/drive/MyDrive/data.csv',encoding = 'unicode_escape')

data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [3]:
data.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom


In [5]:
data.shape

(541909, 8)

In [6]:
data.size

4335272

In [10]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [9]:
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

 ***PREPARATION OF DATA***
 data from united kingdome

In [11]:
uk_data=data[data['Country']=='United Kingdom']
uk_data.shape

(495478, 8)

In [12]:
# removing the items that got canceled
uk_data=uk_data[uk_data['Quantity']>0]
uk_data.shape

(486286, 8)

In [15]:
# dropping null values
uk_data.dropna(subset=['CustomerID'],how='all',inplace=True)
uk_data.shape

(354345, 8)

In [16]:
#check if there is null value or noy
uk_data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [17]:
# keep the data upto one year because it is better to use metric per month or year
uk_data=uk_data[uk_data['InvoiceDate']>= "2010-12-09"]
uk_data.shape

(176137, 8)

In [19]:
# we require only InvoiceNo,StockCode,CustomerID


**Recency**

we need to choose date to evaluate how many days ago was the customor's last purchase

In [20]:
#last date available in our dataset
uk_data['InvoiceDate'].max()

'9/9/2011 9:52'

In [21]:
# new column date
uk_data['date'] = pd.DatetimeIndex(uk_data['InvoiceDate']).date
uk_data.head(10)

<ipython-input-21-db2a08909973>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uk_data['date'] = pd.DatetimeIndex(uk_data['InvoiceDate']).date


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date
105335,545220,21955,DOORMAT UNION JACK GUNS AND ROSES,2,3/1/2011 8:30,7.95,14620.0,United Kingdom,2011-03-01
105336,545220,48194,DOORMAT HEARTS,2,3/1/2011 8:30,7.95,14620.0,United Kingdom,2011-03-01
105337,545220,22556,PLASTERS IN TIN CIRCUS PARADE,12,3/1/2011 8:30,1.65,14620.0,United Kingdom,2011-03-01
105338,545220,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,3/1/2011 8:30,4.95,14620.0,United Kingdom,2011-03-01
105339,545220,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,3/1/2011 8:30,3.75,14620.0,United Kingdom,2011-03-01
105340,545220,21485,RETROSPOT HEART HOT WATER BOTTLE,3,3/1/2011 8:30,4.95,14620.0,United Kingdom,2011-03-01
105341,545220,17091J,VANILLA INCENSE IN TIN,6,3/1/2011 8:30,1.25,14620.0,United Kingdom,2011-03-01
105342,545220,84678,CLASSICAL ROSE SMALL VASE,6,3/1/2011 8:30,2.55,14620.0,United Kingdom,2011-03-01
105343,545220,21700,BIG DOUGHNUT FRIDGE MAGNETS,12,3/1/2011 8:30,0.85,14620.0,United Kingdom,2011-03-01
105344,545220,22191,IVORY DINER WALL CLOCK,2,3/1/2011 8:30,8.50,14620.0,United Kingdom,2011-03-01


In [46]:
#groupby to check last day purchase
recency_data=uk_data.groupby(by='CustomerID', as_index=False)['date'].max()
recency_data.columns=['CustomerID','LastPurshaceDate']
recency_data.head()

,CustomerID,LastPurshaceDate
0,12747.0,2011-08-22
1,12748.0,2011-09-30
2,12749.0,2011-08-01
3,12820.0,2011-09-26
4,12821.0,2011-05-09


In [43]:
# The last date we have is 2011-12-09 so we will use it as reference.
now = dt.date(2011,12,9)
print(now)

2011-12-09


In [47]:
# calculate recency
recency_data['Recency']=recency_data.LastPurshaceDate.apply(lambda x:(now-x).days)
recency_data.head()

,CustomerID,LastPurshaceDate,Recency
0,12747.0,2011-08-22,109
1,12748.0,2011-09-30,70
2,12749.0,2011-08-01,130
3,12820.0,2011-09-26,74
4,12821.0,2011-05-09,214


In [ ]:
recency_data.drop(columns=['LastPurshaceDate'],inplace=True)
recency_data.head()

**Frequency**

how many times a customer purchased from us. To do that we need to check how many invoices are registered by the same customer.

In [50]:
frequency_data=uk_data.copy()
frequency_data.drop_duplicates(subset=['CustomerID','InvoiceNo'], keep="first", inplace=True)
frequency_data=frequency_data.groupby('CustomerID',as_index=False)['InvoiceNo'].count()
frequency_data.columns = ['CustomerID','Frequency']
frequency_data.head()

,CustomerID,Frequency
0,12747.0,5
1,12748.0,96
2,12749.0,3
3,12820.0,1
4,12821.0,1


**Monetary**

How much money did the customer spent over time?

To do that, first, we will create a new column total cost to have the total price per invoice

In [54]:
uk_data['Total_cost'] = uk_data['UnitPrice'] * uk_data['Quantity']

In [58]:
monetary_df = uk_data.groupby(by='CustomerID',as_index=False)['Total_cost'].sum()
monetary_df.columns = ['CustomerID','Monetary']

monetary_df.head()
     

,CustomerID,Monetary
0,12747.0,1760.09
1,12748.0,14680.85
2,12749.0,2755.23
3,12820.0,217.77
4,12821.0,92.72


In [60]:
# now combine recency, frequency and monetary
#combine first recency and frequency..
resfreq = recency_data.merge(frequency_data,left_on='CustomerID',right_on='CustomerID')

#combibe rf frame with monetary values..

RFM = resfreq.merge(monetary_df,left_on='CustomerID',right_on='CustomerID')

RFM.set_index('CustomerID',inplace=True)
RFM.head(10)

,Recency,Frequency,Monetary
CustomerID,,,
12747.0,109,5,1760.09
12748.0,70,96,14680.85
12749.0,130,3,2755.23
12820.0,74,1,217.77
12821.0,214,1,92.72
12822.0,70,2,948.88
12823.0,74,3,1224.00
12826.0,71,2,355.52
12828.0,99,3,415.51
